This notebook executes all data processing and plotting functions that are part of the publication.
It is structured in two blocks, the first one does the data processing, the second step does the plotting. 
Running the first block on your machine might proove difficult if you do not have access to the DKRZ computing infrastructure. 
Therefore, the first block can be bypassed by retrieving the intermediate outputs from zenodo (link) and only running the plotting scripts. 

In [ ]:
# Start a dask client.
from dask.distributed import Client
client = Client(silence_logs=50)
client

In [ ]:
# prevent Figures from showing up here and cluttering this notebook
%matplotlib agg

In [ ]:
import compute_country_aggregates
import compute_signals
import plot_lu_maps
import plot_s10_maps
import plot_s10_scatter
import plot_s10_country_heatmaps
import plot_temperature_gradient
import compute_plot_pattern_correlation

### **Calculations**

In [ ]:
# Compute changes in 10m wind speeds
compute_signals.calculate_signals(time_aggregation="annual", variable_id="sfcWind")
compute_signals.calculate_signals(time_aggregation="monthly", variable_id="sfcWind")
compute_signals.calculate_signals("annual", "sfcWind", full_ensemble=True)

In [ ]:
# compute changes in near-surface air temperature (tas), skin temperature (ts) and sea ice cover (sic)
for variable in ["tas", "ts", "sic"]:
    print(variable)
    compute_signals.calculate_signals(time_aggregation="monthly", variable_id=variable)

In [ ]:
# compute tas changes for full ensemble
compute_signals.calculate_signals("annual", "tas", full_ensemble=True)

In [ ]:
# compute changes in tas - ts, a proxy for stability change
compute_signals.compute_monthly_stability_change()

In [ ]:
# Compute country aggregates
compute_country_aggregates.compute_all(True)  # onshore
compute_country_aggregates.compute_all(False) # offshore

### **Plots**

In [ ]:
# Maps of 10m wind speeds
plot_s10_maps.make_s10_maps()

In [ ]:
plot_s10_maps.make_aggregate_monthly_plots("tas-ts", "mean", ["historical", "rcp45", "rcp85"])

In [ ]:
# Scatter plots of wind change per country
plot_s10_scatter.make_s10_scatter(onshore=False)
plot_s10_scatter.make_s10_scatter(onshore=True)

In [ ]:
# Heatmaps of wind per country
plot_s10_country_heatmaps.make_s10_heatmaps(onshore=False)
plot_s10_country_heatmaps.make_s10_heatmaps(onshore=True)

In [ ]:
# Maps of land use change forcing data
plot_lu_maps.make_LUH_maps()

In [ ]:
# Scatter plot of land use forcing vs wind speed change
compute_plot_pattern_correlation.make_plot()

In [ ]:
# Temperature gradient plots
plot_temperature_gradient.make_all_plots()

In [ ]:
# Shutting down the dask client again
client.close()